following: https://github.com/cathalmccabe/PYNQ_tutorials/blob/master/hls_m_axi/hls_m_axi_example.ipynb

In [8]:
from pynq import Overlay

In [11]:
!ls /home/xilinx/pynq/overlays/

adder	 fir_accel   __init__.py  move_ab	 __pycache__
add_vec  fir_accel2  linreg_vadd  new_adder
base	 fir_accel3  logictools   new_fir_accel


In [12]:
!ls /home/xilinx/pynq/overlays/move_ab

move_ab_1.bit  move_ab_1.hwh  move_ab_1.tcl


In [14]:
overlay = Overlay("/home/xilinx/pynq/overlays/move_ab/move_ab_1.bit")

In [15]:
help(overlay)

Help on Overlay in module pynq.overlay:

<pynq.overlay.Overlay object>
    Default documentation for overlay /home/xilinx/pynq/overlays/move_ab/move_ab_1.bit. The following
    attributes are available on this overlay:
    
    IP Blocks
    ----------
    toplevel_0           : pynq.overlay.DefaultIP
    processing_system7_0 : pynq.overlay.DefaultIP
    
    Hierarchies
    -----------
    None
    
    Interrupts
    ----------
    None
    
    GPIO Outputs
    ------------
    None
    
    Memories
    ------------
    PSDDR                : Memory



In [16]:
tl = overlay.toplevel_0
help(tl)

Help on DefaultIP in module pynq.overlay object:

class DefaultIP(builtins.object)
 |  DefaultIP(description)
 |  
 |  Driver for an IP without a more specific driver
 |  
 |  This driver wraps an MMIO device and provides a base class
 |  for more specific drivers written later. It also provides
 |  access to GPIO outputs and interrupts inputs via attributes. More specific
 |  drivers should inherit from `DefaultIP` and include a
 |  `bindto` entry containing all of the IP that the driver
 |  should bind to. Subclasses meeting these requirements will
 |  automatically be registered.
 |  
 |  Attributes
 |  ----------
 |  mmio : pynq.MMIO
 |      Underlying MMIO driver for the device
 |  _interrupts : dict
 |      Subset of the PL.interrupt_pins related to this IP
 |  _gpio : dict
 |      Subset of the PL.gpio_dict related to this IP
 |  
 |  Methods defined here:
 |  
 |  __init__(self, description)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  read(se

In [17]:
tl.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  a_1 = Register(a=write-only),
  a_2 = Register(a=write-only),
  length_r = Register(length_r=write-only),
  value_r = Register(value_r=write-only)
}

In [18]:
from pynq import allocate
import numpy as np

py_buffer = allocate(shape=(100,), dtype=np.int32)

In [19]:
print("py_buffer physical address {}".format(hex(py_buffer.physical_address)))

py_buffer physical address 0x1684a000


## Configure the IP

Write the values of the memory address, the length, and the value to be added to each location. You must make sure that length doesn't exceed the lendth of the memory that we allocated earlier AND the maximum value that can be set in the HLS IP. The buffe

In [21]:
tl.register_map.a_1 = py_buffer.physical_address
tl.register_map.length_r = 100
tl.register_map.value_r = 7

## Start the IP

In [22]:
tl.register_map.CTRL.AP_START = 1

If we start the IP once, when the IP completes an iteration DONE will go high once. While the IP is working, AP_IDLE will be 0. In this example, the time for an iteration of the HLS IP is very low so you will probably not see AP_IDLE = 0.

Done is cleared when it is read. There is a limitation with the register_map. Multiple reads to the same register may be made to collect the information required for the register_map. This may mean DONE is cleared while the values of some of the other bits and registers are being read and consequentally will not show as high in the register m

In [23]:
tl.register_map.CTRL.AP_DONE

1

In [24]:
tl.register_map.CTRL.AP_IDLE

1

If you have a long running HLS IP, you can poll on the DONE bit before continuing with your application.

## Check the contents of the memory

In [25]:
py_buffer

PynqBuffer([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
            7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
            7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
            7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
            7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7])

In [26]:
len(py_buffer)

100

In [ ]:
my_ip.register_map.CTRL.AUTO_RESTART = 1

In [ ]:
# %xdel in1_vadd
# %xdel in2_vadd
# %xdel out
overlay.free()

# OLD

In [21]:
overlay.scalar_add.add(15, 20)

35

In [8]:
add_ip.write(0x10, 4)
add_ip.write(0x18, 5)
add_ip.read(0x20)

9

In [18]:
from pynq import DefaultIP


class AddDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ["xilinx.com:hls:add:1.0"]

    def add(self, a, b):
        self.write(0x10, a)
        self.write(0x18, b)
        return self.read(0x20)

In [19]:
overlay = Overlay("/home/xilinx/pynq/overlays/adder/adder.bit")

In [20]:
help(overlay)

Help on Overlay in module pynq.overlay:

<pynq.overlay.Overlay object>
    Default documentation for overlay /home/xilinx/pynq/overlays/adder/adder.bit. The following
    attributes are available on this overlay:
    
    IP Blocks
    ----------
    scalar_add           : __main__.AddDriver
    processing_system7_0 : pynq.overlay.DefaultIP
    
    Hierarchies
    -----------
    None
    
    Interrupts
    ----------
    None
    
    GPIO Outputs
    ------------
    None
    
    Memories
    ------------
    PSDDR                : Memory

